## The third In-class-exercise (9/16/2020, 20 points in total)

The purpose of this exercise is to under users' information needs, then collect the data for analysis.

Question 1 (8 points): Describe an interesting research question (or practical question) you have in mind, what kind of data should be collected to answer the question(s)? How many data needed for the analysis? The detail steps for collecting and save the data. 

In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
Research Question: There are lot of news articles websites. My question is how many articles posted in them are informative or opinion-based or emotional content? The expected content from news articles must be mostly informative rather than opinions.
I have chosen Inshorts website for my news articles as they present the news in just 60 words which would be precise in analysing. 
Data required for this analysis is the content of all the articles posted on this website. Also extract titles of the articles for visualization purposes.

Summary of Data Collection:
I have used the BeautifulSoup library to extract the information from the website.
I also have the class names from which data has to be taken. 
As the website has jquery based listing of articles. I had to raise continuous requests with updated post-ids to get the next post data.
I used json and re for json formatting and regex to identify post-id

Steps of Data Collection
1. Request for articles data from url using requests and get the response
2. Extract the required data from the articles using BeautifulSoap from the first page
3. Based on min-news-id extract get data of next article and continue the same until 500 articles are extracted





'''

Question 2 (12 points): Write python code to collect 500 items of the data you plan to collect above.

In [2]:
# You code here (Please add comments in the code):

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json

def extractData(data): # Extracts title and contents of article from html data
  soup = BeautifulSoup(data, 'html.parser')
  headline_list = []
  body_list = []
  all_news = soup.findAll("div",{"class":"news-card z-depth-1"})
  count = len(all_news)
  for each_news in all_news:
    article_body = each_news.find("div", {"itemprop" : "articleBody"}).get_text()
    headline = each_news.find("span", {"itemprop" : "headline"}).get_text()
    headline_list.append(headline)
    body_list.append(article_body)
  
  return headline_list, body_list

def get_headers(): # The headers for the http request
    return {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-IN,en-US;q=0.9,en;q=0.8",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "cookie": "_ga=GA1.2.474379061.1548476083; _gid=GA1.2.251903072.1548476083; __gads=ID=17fd29a6d34048fc:T=1548476085:S=ALNI_MaRiLYBFlMfKNMAtiW0J3b_o0XGxw",
        "origin": "https://inshorts.com",
        "referer": "https://inshorts.com/en/read/",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "x-requested-with": "XMLHttpRequest"
    }

title_text = [] # List to store News Titles
content_text =[] #List to store News Content

temp_title = []
temp_content = []

url = "https://www.inshorts.com/en/read"
request = requests.get(url)
title_text, content_text = extractData(request.text)  # Extract articles in First page - 25 articles

regex_min_news_id = re.compile('var min_news_id = "(.*?)";')
min_news_id = regex_min_news_id.search(request.text).group(1)

for number in range(475): # Extract 475 more articles
  ajax_url = 'https://inshorts.com/en/ajax/more_news'
  response = requests.post(ajax_url, data={"category": "", "news_offset": min_news_id}, headers=get_headers())
  response_json = json.loads(response.text)
  temp_title, temp_content = extractData(response_json["html"])
  title_text.append(temp_title)
  content_text.append(temp_content)
  temp_title = []
  temp_content = []
  min_news_id = response_json["min_news_id"]

df = pd.DataFrame(list(zip(title_text, content_text)), columns =['Article Title', 'Article Content'])  # Creating Dataframe
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 500


,Article Title,Article Content
0,"Entry, exit gates of 8 Delhi Metro stations cl...",Entry and exit gates of Delhi Metro stations i...
1,He had a rape room in his house: Singer Phoebe...,Singer Phoebe Bridgers has said musician Maril...
2,"India reports 11,713 new COVID-19 cases, 95 de...","India has reported 11,713 new COVID-19 cases a..."
3,Understanding COVID-19 origins will take years...,A member of the World Health Organization team...
4,Stuart Broad mocks ICC for praising Virat Kohl...,"Pacer Stuart Broad, who doesn't feature in Eng..."
...,...,...
495,[Ghazipur border on NH 24 closed for Ghaziabad...,[Ghazipur border on NH 24 is closed for traffi...
496,[BCCI to discuss adding 2 new teams to IPL in ...,[The BCCI will discuss adding two teams to IPL...
497,[Who are India's richest women & what are thei...,[HCL Tech's Chairperson Roshni Nadar Malhotra ...
498,"[SA lottery probed after 5,6,7,8, 9 & 10 drawn...",[South Africa's national lottery is being prob...
